## Setup

In [1]:
import sys, time
sys.path.insert(0, "../src/")
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
from matplotlib import pyplot as plt
import torch
from torch import nn as nn
from chexnet_medical_ai.image_preprocessing import ImageTransForm, ImageDataLoader
from chexnet_medical_ai import model

## Train and Test Loaders 

In [3]:
train_path = "/mnt/d/archive/train/"
test_path = "/mnt/d/archive/test/"

In [4]:
train_transform = ImageTransForm().train_image_transforms()
test_transform = ImageTransForm().test_image_transforms()

In [5]:
train_data = ImageDataLoader(train_path, test_path).train_data(train_transform)
test_data = ImageDataLoader(train_path, test_path).test_data(test_transform)

In [6]:
train_data.classes, test_data.classes

(['0', '1'], ['0', '1'])

In [7]:
torch.manual_seed(42)
train_loader = ImageDataLoader(train_path, test_path).train_data_loader(train_transform)
test_loader = ImageDataLoader(train_path, test_path).test_data_loader(test_transform)

## Model

In [8]:
torch.manual_seed(101)
cnn_model = model.ConvolutionalNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

In [9]:
start_time = time.time()

epochs = 10
max_train_batch = 800
max_test_batch = 300

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):

    trn_corr = 0
    tst_corr = 0


    for b, (X_train, y_train) in enumerate(train_loader):

        if b == max_train_batch:
            break

        y_pred = cnn_model(X_train)
        loss = criterion(y_pred, y_train)

        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if b % 200 == 0:
            print(f"Epoch {i+1}/{epochs} Batch {b} Loss: {loss.item():.4f}")

    train_losses.append(loss.item())
    train_correct.append(trn_corr)


    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):

            if b == max_test_batch:
                break

            y_val = cnn_model(X_test)
            predicted = torch.max(y_val.data, 1)[1]
            batch_corr = (predicted == y_test).sum()
            tst_corr += batch_corr

    loss = criterion(y_val, y_test)
    test_losses.append(loss.item())
    test_correct.append(tst_corr)

total_time = time.time() - start_time
print(f"Training completed in {total_time/60:.2f} minutes")


Epoch 1/10 Batch 0 Loss: 0.6849
Epoch 1/10 Batch 200 Loss: 0.0137
Epoch 1/10 Batch 400 Loss: 0.0324
Epoch 1/10 Batch 600 Loss: 0.0498
Epoch 2/10 Batch 0 Loss: 0.0055
Epoch 2/10 Batch 200 Loss: 0.0241
Epoch 2/10 Batch 400 Loss: 0.0034
Epoch 2/10 Batch 600 Loss: 0.0145
Epoch 3/10 Batch 0 Loss: 0.5701
Epoch 3/10 Batch 200 Loss: 0.0007
Epoch 3/10 Batch 400 Loss: 0.0072
Epoch 3/10 Batch 600 Loss: 0.0008
Epoch 4/10 Batch 0 Loss: 0.0027
Epoch 4/10 Batch 200 Loss: 0.0171
Epoch 4/10 Batch 400 Loss: 0.0017
Epoch 4/10 Batch 600 Loss: 0.0094
Epoch 5/10 Batch 0 Loss: 0.0091
Epoch 5/10 Batch 200 Loss: 0.0119
Epoch 5/10 Batch 400 Loss: 0.0080
Epoch 5/10 Batch 600 Loss: 0.0034
Epoch 6/10 Batch 0 Loss: 0.0159
Epoch 6/10 Batch 200 Loss: 0.0023
Epoch 6/10 Batch 400 Loss: 0.0209
Epoch 6/10 Batch 600 Loss: 0.0077
Epoch 7/10 Batch 0 Loss: 0.0124
Epoch 7/10 Batch 200 Loss: 0.0270
Epoch 7/10 Batch 400 Loss: 0.0099
Epoch 7/10 Batch 600 Loss: 0.0146
Epoch 8/10 Batch 0 Loss: 0.0066
Epoch 8/10 Batch 200 Loss: 0.0